In [1]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama',
)

In [2]:
from neo4j import GraphDatabase

# Define connection details
URI = "bolt://localhost:7687"  # Neo4j Bolt connection
USERNAME = "neo4j"
PASSWORD = "password"  # Replace with the password you set

# Create a Neo4j Driver instance
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Define a function to test the connection
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connected to Neo4j' AS message")
        for record in result:
            print(record["message"])

# Run test
test_connection()

# Close the driver when done
#driver.close()

Connected to Neo4j


In [3]:
import pandas as pd

# Define the file path
file_path = "IPR Avezzano Indexes 1809-1865.xlsx"

# Load the specific sheet "Marriages 1821 - 1845"
df = pd.read_excel(file_path, sheet_name="Marriages 1821-1845").head(15)

# Correct mother name

In [4]:
import pandas as pd
import re

def format_proper_case(name):
    """
    ✅ Converts all words to start with a capital letter.
    - Fixes "di", "del", "de".
    - Removes "fu" / "fù" and detects if the person was deceased.
    """
    if not isinstance(name, str) or name.strip() == "":
        return name, ""  # ✅ Return unchanged name & empty deceased status

    # ✅ Detect "fu" / "fù" (deceased)
    deceased = ""
    match = re.match(r"^(fu|fù)\s*", name, re.IGNORECASE)
    
    if match:
        deceased = "before {date}"  # 🔹 Placeholder (updated later)
        name = name[len(match.group(0)):].strip()  # ✅ Remove "fu"/"fù"

    # ✅ Fix "di", "del", "de" capitalization
    name = re.sub(r"\b(di|del|de)(?=\s|\b)", lambda x: x.group(1).capitalize(), name, flags=re.IGNORECASE)

    # ✅ Capitalize all words
    name = " ".join(word.capitalize() for word in name.split())

    return name, deceased  # ✅ Return formatted name + deceased status


def format_deceased_status(name, date, year):
    """
    ✅ If "fu" / "fù" is present, mark person as deceased **before the marriage date**.
    ✅ If not, return an empty string.
    ✅ Fixes Timestamp issue by formatting the date properly.
    """
    formatted_name, deceased = format_proper_case(name)  # ✅ Format name + detect "fu"

    # ✅ Ensure date is properly converted to a string
    if deceased and pd.notna(date) and pd.notna(year):
        date_str = date.strftime("%d %b") if isinstance(date, pd.Timestamp) else str(date).strip()
        year_str = str(year).strip()
        deceased = f"before {date_str} {year_str}"  # ✅ Correct format: "before 09 Dec 1821"

    return formatted_name, deceased


# ✅ Work on a **COPY** of df (DO NOT modify the original)
df_corrected = df.copy()

# ✅ Strip spaces from column names
df_corrected.columns = df_corrected.columns.str.strip()

# ✅ Define columns to fix
columns_to_fix = [
    "Last name - Groom", "First name - Groom", "Father of Groom", "Mother of Groom",
    "Last name - Bride", "First name - Bride", "Father of Bride", "Mother of Bride",
    "Comune Groom", "Comune Bride"
]

# ✅ Apply transformations
for col in columns_to_fix:
    if col in df_corrected.columns:
        df_corrected[[col, f"Deceased - {col}"]] = df_corrected.apply(
            lambda row: format_deceased_status(row[col], row["Date"], row["Year"]), axis=1, result_type="expand"
        )

# ✅ Save Corrected Data (Safe Copy)
df_corrected.to_csv("corrected_marriage_records.csv", index=False)

print("✅ Successfully capitalized names, handled 'fu', fixed date issues, and saved deceased statuses!")

✅ Successfully capitalized names, handled 'fu', fixed date issues, and saved deceased statuses!


In [7]:
df_corrected.head()

,Record #,Last name - Groom,First name - Groom,Father of Groom,Mother of Groom,Last name - Bride,First name - Bride,Father of Bride,Mother of Bride,Date,...,Deceased - Last name - Groom,Deceased - First name - Groom,Deceased - Father of Groom,Deceased - Mother of Groom,Deceased - Last name - Bride,Deceased - First name - Bride,Deceased - Father of Bride,Deceased - Mother of Bride,Deceased - Comune Groom,Deceased - Comune Bride
0,3,Iannotti,Ascenzo,Giuseppe,Teresa Buttari,De Santis,Anna Berardina,Nicola,Maddalena Del Rosso,2020-07-30,...,,,,,,,,,,
1,6,Lolli,Anselmo,Andrea,Camilla Liberati,Collalto,Anna Giuseppa,Gabrielefu,Elisabetta Savina,2020-12-09,...,,,before 09 Dec 1821,,,,,before 09 Dec 1821,,
2,2,Paciotti,Francesco,Pietro,Giacinta Di Simone,Lolli,Maria,Costanzo,Domenica Colella,2020-05-20,...,,,,,,,,,,
3,1,Pennazza,Giuseppe,Gennaro,Gesualda Di Clemente,Novelli,Marianna,Francesco,Maria Domenica Marianella,2020-01-21,...,,,,,,,,,,
4,5,Rodorigo,Giacomo,Felice,Rosa Di Crescenzo,Donsante,Aurora,Arcangelo,Clementina Cosimo,2020-11-25,...,,,,,,,,,,


# Check first name and last name for mothers with LLM

In [10]:
from openai import OpenAI
import pandas as pd
import json
from pydantic import BaseModel

# ✅ Setup DeepSeek LLM
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# ✅ Define Pydantic Model for Structured Response
class NameSplit(BaseModel):
    firstname: str
    lastname: str

# ✅ Function to Generate LLM Prompt
def generate_name_split_prompt(full_name):
    """
    Generates a structured prompt for the LLM to split names.
    - Keeps **full names intact** (no missing middle names).
    - Uses structured JSON output via Pydantic.
    """
    return f"""
    You are an expert in **historical genealogical records** in Italian.
    Your task is to **correctly split Italian full names of mothers** into:
    
    - **First Name:** The complete first name (including double names if present).
    - **Last Name:** The proper last name (with correct capitalization).

    📌 **Rules:**
    - **Do not change spelling** of any names.
    - Ensure the **full name remains intact**.
    - Format output as JSON with `"firstname"`, `"lastname"`.

    ---
    **Example Inputs → Outputs**
    - `"Maria Domenica Marianella"` → `{{
        "firstname": "Maria Domenica",
        "lastname": "Marianella"
    }}`
    - `"Rosa De Luca"` → `{{
        "firstname": "Rosa",
        "lastname": "De Luca"
    }}`

    ---
    🔍 **Now split the following name correctly:**
    **Input:** `{full_name}`
    **Output (JSON only):**
    """

# ✅ Function to Process Mother Names using LLM
def process_mother_names(row):
    """
    Uses the LLM to correct 'Mother of Groom' and 'Mother of Bride' names,
    and splits them into first and last names.
    """
    corrected_data = {}

    for col in ["Mother of Groom", "Mother of Bride"]:
        full_name = row[col].strip() if pd.notna(row[col]) else ""

        if not full_name or " " not in full_name:  # Skip if no valid name
            corrected_data[col] = {"firstname": full_name, "lastname": ""}
            continue

        # ✅ Call LLM using `.parse()` for structured response
        try:
            response = client.beta.chat.completions.parse(
                model="mistral:7b",
                temperature=0,
                messages=[{"role": "user", "content": generate_name_split_prompt(full_name)}],
                response_format=NameSplit,  # ✅ Structured Pydantic Response
            )

            # ✅ Extract structured data
            if response.choices[0].message.parsed:
                parsed_data = response.choices[0].message.parsed
                corrected_data[col] = {
                    "firstname": parsed_data.firstname.strip(),
                    "lastname": parsed_data.lastname.strip()
                }
                print(f"✅ {full_name} → First name: {parsed_data.firstname}, Last name: {parsed_data.lastname}")

            else:
                print(f"⚠️ LLM Refused or Failed for: {full_name}")
                corrected_data[col] = {"firstname": full_name, "lastname": ""}

        except Exception as e:
            print(f"❌ Error processing '{full_name}': {e}")
            corrected_data[col] = {"firstname": full_name, "lastname": ""}  # Keep fallback

    return corrected_data

# ✅ Work on a **COPY** of df_corrected → Output will be df_llm
df_llm = df_corrected.copy()

# ✅ Apply LLM Correction Row by Row
corrected_mother_names = df_llm.apply(process_mother_names, axis=1)

# ✅ Convert corrected data into separate columns
for index, corrections in corrected_mother_names.items():
    for col, values in corrections.items():
        df_llm.loc[index, f"First name - {col}"] = values["firstname"]
        df_llm.loc[index, f"Last name - {col}"] = values["lastname"]

# ✅ Save Corrected Data (Safe Copy)
df_llm.to_csv("corrected_marriage_records_llm.csv", index=False)

print("✅ Successfully corrected mother names using LLM and saved the new dataset!")

✅ Teresa Buttari → First name: Teresa, Last name: Buttari
✅ Maddalena Del Rosso → First name: Maddalena, Last name: Del Rosso
✅ Camilla Liberati → First name: Camilla, Last name: Liberati
✅ Elisabetta Savina → First name: Elisabetta, Last name: Savina
✅ Giacinta Di Simone → First name: Giacinta, Last name: Di Simone
✅ Domenica Colella → First name: Domenica, Last name: Colella
✅ Gesualda Di Clemente → First name: Gesualda, Last name: Di Clemente
✅ Maria Domenica Marianella → First name: Maria Domenica, Last name: Marianella
✅ Rosa Di Crescenzo → First name: Rosa, Last name: Di Crescenzo
✅ Clementina Cosimo → First name: Clementina, Last name: Cosimo
✅ Vittoria Gatto → First name: Vittoria, Last name: Gatto
✅ Felicia Di Pietro → First name: Felicia, Last name: Di Pietro
✅ Caterina Iacoboni → First name: Caterina, Last name: Iacoboni
✅ Gesualda Di Clemente → First name: Gesualda, Last name: Di Clemente
✅ Maria Felice Paciotti → First name: Maria Felice, Last name: Paciotti
✅ Maria Sorge 

In [9]:
# TO DO:
# - adapt the rest of the code to the entries with firstName and lastName
# - optimize the rules by checking for any () inside the people name and create the field "alias lastname" and "alias name"

In [6]:
df_llm.head()

,Record #,Last name - Groom,First name - Groom,Father of Groom,Mother of Groom,Last name - Bride,First name - Bride,Father of Bride,Mother of Bride,Date,...,Deceased - Last name - Bride,Deceased - First name - Bride,Deceased - Father of Bride,Deceased - Mother of Bride,Deceased - Comune Groom,Deceased - Comune Bride,First name - Mother of Groom,Last name - Mother of Groom,First name - Mother of Bride,Last name - Mother of Bride
0,3,Iannotti,Ascenzo,Giuseppe,Teresa Buttari,De Santis,Anna Berardina,Nicola,Maddalena Del Rosso,2020-07-30,...,,,,,,,Teresa,Buttari,Maddalena,Del Rosso
1,6,Lolli,Anselmo,Andrea,Camilla Liberati,Collalto,Anna Giuseppa,Gabrielefu,Elisabetta Savina,2020-12-09,...,,,,before 09 Dec 1821,,,Camilla,Liberati,Elisabetta,Savina
2,2,Paciotti,Francesco,Pietro,Giacinta Di Simone,Lolli,Maria,Costanzo,Domenica Colella,2020-05-20,...,,,,,,,Giacinta,Di Simone,Domenica,Colella
3,1,Pennazza,Giuseppe,Gennaro,Gesualda Di Clemente,Novelli,Marianna,Francesco,Maria Domenica Marianella,2020-01-21,...,,,,,,,Gesualda,Di Clemente,Maria Domenica,Marianella
4,5,Rodorigo,Giacomo,Felice,Rosa Di Crescenzo,Donsante,Aurora,Arcangelo,Clementina Cosimo,2020-11-25,...,,,,,,,Rosa,Di Crescenzo,Clementina,Cosimo


# Create relationships, entities

In [15]:
from pydantic import BaseModel
from typing import List, Optional
import pandas as pd
import json
import numpy as np
import re
from neo4j import GraphDatabase

# ✅ Define Schema for Persons (Includes 'deceased' property)
class Person(BaseModel):
    fullname: str
    firstname: str
    lastname: str
    gender: str
    location: Optional[str] = None
    deceased: Optional[str] = ""  # ✅ Always present, empty by default

# ✅ Define Schema for Relationships
class Relationship(BaseModel):
    from_person: str
    to_person: str
    type: str
    date: Optional[str] = None

# ✅ Define Full Marriage Record Schema
class MarriageRecord(BaseModel):
    persons: List[Person]
    relationships: List[Relationship]

# ✅ Function to Format Date Correctly
def format_date(date_value, year_value):
    """✅ Ensures the date is always in 'dd MMM yyyy' format."""
    if isinstance(date_value, pd.Timestamp):
        date_value = date_value.strftime("%d %b")  # Convert Timestamp
    elif isinstance(date_value, str):
        date_value = date_value.strip()  # Clean up strings

    year_value = str(year_value).strip() if pd.notna(year_value) else ""

    return f"{date_value} {year_value}" if date_value and year_value else None

# ✅ Function to Retrieve Deceased Status from df_llm
def get_deceased_status(row, person_type):
    """✅ Retrieves correct deceased status from df_llm based on person type."""
    deceased_col = f"Deceased - {person_type}"
    return row[deceased_col] if deceased_col in row and pd.notna(row[deceased_col]) else ""

# ✅ Process Each Marriage Record Row
def process_row(row):
    """Processes a marriage record row into structured JSON."""

    # ✅ Extract relevant fields & Strip whitespace
    groom_firstname = row["First name - Groom"].strip()
    groom_lastname = row["Last name - Groom"].strip()
    bride_firstname = row["First name - Bride"].strip()
    bride_lastname = row["Last name - Bride"].strip()

    father_groom_firstname = row["Father of Groom"].strip()
    mother_groom_firstname = row["First name - Mother of Groom"].strip()  # ✅ Now from df_llm
    mother_groom_lastname = row["Last name - Mother of Groom"].strip()  # ✅ Now from df_llm

    father_bride_firstname = row["Father of Bride"].strip()
    mother_bride_firstname = row["First name - Mother of Bride"].strip()  # ✅ Now from df_llm
    mother_bride_lastname = row["Last name - Mother of Bride"].strip()  # ✅ Now from df_llm

    # ✅ Fix Date Extraction: Convert Timestamp to Proper Format
    date = format_date(row["Date"], row["Year"])

    # ✅ Handle NaN values for location
    comune_groom = row["Comune Groom"]
    comune_bride = row["Comune Bride"]

    comune_groom = None if isinstance(comune_groom, float) and np.isnan(comune_groom) else comune_groom
    comune_bride = None if isinstance(comune_bride, float) and np.isnan(comune_bride) else comune_bride

    # ✅ Get Deceased Status for All People
    deceased_groom = get_deceased_status(row, "First name - Groom")
    deceased_father_groom = get_deceased_status(row, "Father of Groom")
    deceased_mother_groom = get_deceased_status(row, "Mother of Groom")

    deceased_bride = get_deceased_status(row, "First name - Bride")
    deceased_father_bride = get_deceased_status(row, "Father of Bride")
    deceased_mother_bride = get_deceased_status(row, "Mother of Bride")

    # ✅ Create Persons with Deceased Info
    groom = Person(fullname=f"{groom_firstname} {groom_lastname}", firstname=groom_firstname, lastname=groom_lastname, gender="male", location=comune_groom, deceased=deceased_groom)
    bride = Person(fullname=f"{bride_firstname} {bride_lastname}", firstname=bride_firstname, lastname=bride_lastname, gender="female", location=comune_bride, deceased=deceased_bride)

    father_groom = Person(fullname=f"{father_groom_firstname} {groom_lastname}", firstname=father_groom_firstname, lastname=groom_lastname, gender="male", deceased=deceased_father_groom)
    mother_groom = Person(fullname=f"{mother_groom_firstname} {mother_groom_lastname}", firstname=mother_groom_firstname, lastname=mother_groom_lastname, gender="female", deceased=deceased_mother_groom)

    father_bride = Person(fullname=f"{father_bride_firstname} {bride_lastname}", firstname=father_bride_firstname, lastname=bride_lastname, gender="male", deceased=deceased_father_bride)
    mother_bride = Person(fullname=f"{mother_bride_firstname} {mother_bride_lastname}", firstname=mother_bride_firstname, lastname=mother_bride_lastname, gender="female", deceased=deceased_mother_bride)

    persons = [groom, bride, father_groom, mother_groom, father_bride, mother_bride]

    # ✅ Create Relationships (Includes Marriage Date + Parents' Marriage)
    relationships = [
        Relationship(from_person=groom.fullname, to_person=bride.fullname, type="MARRIED_TO", date=date),
        Relationship(from_person=bride.fullname, to_person=groom.fullname, type="MARRIED_TO", date=date),
        Relationship(from_person=groom.fullname, to_person=father_groom.fullname, type="SON_OF"),
        Relationship(from_person=groom.fullname, to_person=mother_groom.fullname, type="SON_OF"),
        Relationship(from_person=bride.fullname, to_person=father_bride.fullname, type="DAUGHTER_OF"),
        Relationship(from_person=bride.fullname, to_person=mother_bride.fullname, type="DAUGHTER_OF"),
        Relationship(from_person=father_groom.fullname, to_person=mother_groom.fullname, type="MARRIED_TO", date=""),
        Relationship(from_person=mother_groom.fullname, to_person=father_groom.fullname, type="MARRIED_TO", date=""),
        Relationship(from_person=father_bride.fullname, to_person=mother_bride.fullname, type="MARRIED_TO", date=""),
        Relationship(from_person=mother_bride.fullname, to_person=father_bride.fullname, type="MARRIED_TO", date=""),
    ]

    return MarriageRecord(persons=persons, relationships=relationships)

# ✅ Process DataFrame from df_llm
records = [process_row(row).model_dump() for _, row in df_llm.iterrows()]

# ✅ Save JSON output
with open("neo4j_data.json", "w") as f:
    json.dump(records, f, indent=2)

print("✅ Successfully processed marriage records into Neo4j-compatible JSON!")

# ✅ Insert into Neo4j
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def insert_marriage_record(tx, record):
    """Inserts a single marriage record into Neo4j with parents' marriages."""
    for person in record["persons"]:
        query = """
        MERGE (p:Person {fullname: $fullname})
        SET p.firstname = $firstname,
            p.lastname = $lastname,
            p.gender = $gender,
            p.location = $location,
            p.deceased = COALESCE(NULLIF(p.deceased, ''), $deceased)
        """
        tx.run(query, **person)

    for relationship in record["relationships"]:
        query = f"""
        MATCH (a:Person {{fullname: $from_person}})
        MATCH (b:Person {{fullname: $to_person}})
        MERGE (a)-[r:{relationship["type"]}]->(b)
        SET r.date = $date
        """
        tx.run(query, **relationship)

with driver.session() as session:
    for record in records:
        session.execute_write(insert_marriage_record, record)

print("✅ Successfully inserted all marriage records into Neo4j with parents' marriages!")

✅ Successfully processed marriage records into Neo4j-compatible JSON!
✅ Successfully inserted all marriage records into Neo4j with parents' marriages!


In [ ]:
# MATCH (n) DETACH DELETE n

# MATCH (n) OPTIONAL MATCH (n)-[r]-() RETURN n, r;